In [14]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split


file_path = 'FS8_PPGUID.csv' 
df = pd.read_csv(file_path, low_memory=False)
df.drop('Unnamed: 24',axis=1, inplace=True)
df['TargetFilename'] = df['TargetFilename'].apply(lambda x: 0 if pd.isnull(x) else len(str(x)))
from sklearn.preprocessing import LabelEncoder
df['LogonId'] = df['LogonId'].fillna(-1)
df['TerminalSessionId'] = df['TerminalSessionId'].fillna(-1)

label_encoder_logon = LabelEncoder()
label_encoder_terminal = LabelEncoder()

df['LogonId'] = df['LogonId'].apply(lambda x: x if x == -1 else label_encoder_logon.fit_transform([x])[0])
df['TerminalSessionId'] = df['TerminalSessionId'].apply(lambda x: x if x == -1 else label_encoder_terminal.fit_transform([x])[0])

df['LogonId'] = df['LogonId'].astype(int)
df['TerminalSessionId'] = df['TerminalSessionId'].astype(int)
df_new = df.drop_duplicates()
df_new.replace('-', 'null', inplace=True)

df_new['CurrentDirectory'] = df_new['CurrentDirectory'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else len(str(x)))
df_new['LogonGuid'] = df_new['LogonGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)


df_new['IntegrityLevel'] = df_new['LogonGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['ParentUser'] = df_new['ParentUser'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['Protocol'] = df_new['Protocol'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['Initiated'] = df_new['Initiated'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourceIsIpv6'] = df_new['SourceIsIpv6'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourceIp'] = df_new['SourceIp'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['DestinationIsIpv6'] = df_new['DestinationIsIpv6'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['DestinationIp'] = df_new['DestinationIp'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['ParentProcessGuid'] = df_new['ParentProcessGuid'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['SourcePort'] = df_new['SourcePort'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else x)
df_new['DestinationPort'] = df_new['DestinationPort'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else x)
df_new['User'] = df_new['User'].apply(lambda x: 0 if pd.isna(x) or x == 'null' else 1)
df_new['label'].value_counts()
df_final = df_new
X = df_final.drop(columns=['label'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_benign = X_scaled[df_new['label'] == 0]
X_malicious = X_scaled[df_new['label'] == 1]

X_train_benign, X_test_benign = train_test_split(X_benign, test_size=0.2, random_state=42)


iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
iso_forest.fit(X_train_benign)


#X_pca_full = pca.transform(X_scaled)


y_test_combined = df_new['label']  


y_pred_iso = iso_forest.predict(X_scaled)
y_pred_iso = [1 if pred == -1 else 0 for pred in y_pred_iso]  


print("Confusion Matrix (Isolation Forest):\n", confusion_matrix(y_test_combined, y_pred_iso))
print("Classification Report (Isolation Forest):\n", classification_report(y_test_combined, y_pred_iso))

C:\Users\User\AppData\Local\Temp\ipykernel_4344\1839821145.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.replace('-', 'null', inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_4344\1839821145.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['CurrentDirectory'] = df_new['CurrentDirectory'].apply(lambda x: -1 if pd.isna(x) or x == 'null' else len(str(x)))
C:\Users\User\AppData\Local\Temp\ipykernel_4344\1839821145.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Confusion Matrix (Isolation Forest):
 [[124722 160204]
 [ 16939  46174]]
Classification Report (Isolation Forest):
               precision    recall  f1-score   support

           0       0.88      0.44      0.58    284926
           1       0.22      0.73      0.34     63113

    accuracy                           0.49    348039
   macro avg       0.55      0.58      0.46    348039
weighted avg       0.76      0.49      0.54    348039



In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 25 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   EventID              1048575 non-null  int64  
 1   Version              1048575 non-null  int64  
 2   Level                1048575 non-null  int64  
 3   Task                 1048575 non-null  int64  
 4   EventRecordID        1048575 non-null  int64  
 5   Execution_ProcessID  1048575 non-null  int64  
 6   Execution_ThreadID   1048575 non-null  int64  
 7   User                 1048574 non-null  object 
 8   label                1048575 non-null  int64  
 9   TargetFilename       1048575 non-null  int64  
 10  CurrentDirectory     185277 non-null   object 
 11  LogonGuid            185277 non-null   object 
 12  LogonId              1048575 non-null  int32  
 13  TerminalSessionId    1048575 non-null  int32  
 14  IntegrityLevel       185277 non-null   object 
 15

In [5]:
import numpy as np

X_test_malicious_sampled = X_malicious[:20000]

X_test_combined = np.concatenate([X_test_benign, X_test_malicious_sampled], axis=0)

y_test_combined = np.array([0] * len(X_test_benign) + [1] * len(X_test_malicious_sampled))
print("y_test_combined labels:\n", y_test_combined)

print("First 20 labels of y_test_combined:\n", y_test_combined[:20])

unique, counts = np.unique(y_test_combined, return_counts=True)
print("Label counts using np.unique:")
print(dict(zip(unique, counts)))

from collections import Counter
label_counts = Counter(y_test_combined)
print("Label counts using Counter:")
print(label_counts)

y_test_combined labels:
 [0 0 0 ... 1 1 1]
First 20 labels of y_test_combined:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Label counts using np.unique:
{0: 56986, 1: 20000}
Label counts using Counter:
Counter({0: 56986, 1: 20000})


In [6]:
X.info()df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348039 entries, 0 to 1048574
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   EventID              348039 non-null  int64  
 1   Version              348039 non-null  int64  
 2   Level                348039 non-null  int64  
 3   Task                 348039 non-null  int64  
 4   EventRecordID        348039 non-null  int64  
 5   Execution_ProcessID  348039 non-null  int64  
 6   Execution_ThreadID   348039 non-null  int64  
 7   User                 348038 non-null  object 
 8   TargetFilename       348039 non-null  int64  
 9   CurrentDirectory     158310 non-null  object 
 10  LogonGuid            158310 non-null  object 
 11  LogonId              348039 non-null  int32  
 12  TerminalSessionId    348039 non-null  int32  
 13  IntegrityLevel       158310 non-null  object 
 14  ParentUser           158310 non-null  object 
 15  Protocol             

In [8]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 348039 entries, 0 to 1048574
Data columns (total 25 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   EventID              348039 non-null  int64  
 1   Version              348039 non-null  int64  
 2   Level                348039 non-null  int64  
 3   Task                 348039 non-null  int64  
 4   EventRecordID        348039 non-null  int64  
 5   Execution_ProcessID  348039 non-null  int64  
 6   Execution_ThreadID   348039 non-null  int64  
 7   User                 348038 non-null  object 
 8   label                348039 non-null  int64  
 9   TargetFilename       348039 non-null  int64  
 10  CurrentDirectory     158310 non-null  object 
 11  LogonGuid            158310 non-null  object 
 12  LogonId              348039 non-null  int32  
 13  TerminalSessionId    348039 non-null  int32  
 14  IntegrityLevel       158310 non-null  object 
 15  ParentUser           

In [9]:
iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
iso_forest.fit(X_train_benign)

y_pred_isf = iso_forest.predict(X_test_combined)
y_pred_isf = np.array([1 if pred == -1 else 0 for pred in y_pred_isf])  

print("Confusion Matrix (Isolation Forest):\n", confusion_matrix(y_test_combined, y_pred_isf))
print("Classification Report (Isolation Forest):\n", classification_report(y_test_combined, y_pred_isf))

Confusion Matrix (Isolation Forest):
 [[24914 32072]
 [ 3652 16348]]
Classification Report (Isolation Forest):
               precision    recall  f1-score   support

           0       0.87      0.44      0.58     56986
           1       0.34      0.82      0.48     20000

    accuracy                           0.54     76986
   macro avg       0.60      0.63      0.53     76986
weighted avg       0.73      0.54      0.56     76986

